In [2]:
import dt_apriltags as atag
import cv2
import numpy as np

In [3]:
# create detector
detector = atag.Detector(families="tagStandard41h12")

In [4]:
calibration_images = [
    # "testimages/cal1.jpg",
    "testimages/cal2.jpg",
    "testimages/cal3.jpg",
    "testimages/cal4.jpg",
    "testimages/cal5.jpg",
    "testimages/cal6.jpg",
    "testimages/cal7.jpg",
    "testimages/cal8.jpg",
    "testimages/cal9.jpg",
    "testimages/cal10.jpg",
    "testimages/cal11.jpg",
    "testimages/cal12.jpg",
    "testimages/cal13.jpg",
    "testimages/cal14.jpg",
    "testimages/cal15.jpg",
    "testimages/cal16.jpg",
]

calibration_grid = (9, 5)
calibration_sep = (0.25, 0.5)
calibration_size = 0.75
calibration_offset = (
    calibration_sep[0] + calibration_size,
    calibration_sep[1] + calibration_size,
)
pixel_per_tag = 9
pixel_width = calibration_size / pixel_per_tag
tag_count = calibration_grid[0] * calibration_grid[1]
tag_corners = [
    [2 * pixel_width, 2 * pixel_width],
    [2 * pixel_width, 7 * pixel_width],
    [7 * pixel_width, 7 * pixel_width],
    [7 * pixel_width, 2 * pixel_width],
]

In [5]:
truepoints = []
for x in range(calibration_grid[0]):
    for y in range(calibration_grid[1]):
        origin = (x * calibration_offset[0], y * calibration_offset[1])
        for corner in tag_corners:
            truepoints.append([origin[0] + corner[0], origin[1] + corner[1], 0])


xgrid = np.array([p[0] for p in truepoints])
ygrid = np.array([p[1] for p in truepoints])
zgrid = np.array([p[2] for p in truepoints])

opoints = np.dstack((xgrid, ygrid, zgrid)).reshape((-1, 1, 3)).astype(np.float32)

In [31]:
imagesize = None

truepoints = []
ipoints = []

usableImageCount = 0
unusable = []

for filename in calibration_images:
    rgb = cv2.imread(filename)

    if rgb is None:
        print("warning: error opening {}, skipping".format(filename))
        continue

    cursize = (rgb.shape[1], rgb.shape[0])

    if imagesize is None:
        imagesize = cursize
    else:
        assert imagesize == cursize

    print("loaded " + filename + " of size {}x{}".format(*imagesize))

    if len(rgb.shape) == 3:
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    else:
        gray = rgb

    results = detector.detect(gray)
    if len(results) == tag_count:
        usableImageCount += 1
        truepoints.append(opoints)
        foundpoints = []
        for tag in results:
            corners = tag.corners
            for c in corners:
                foundpoints.append([c])

        ipoints.append(foundpoints)
    else:
        print("could not find all apriltags")
        unusable.append(str(len(results)) + "/" + str(tag_count) + " " + filename)


flags = (
    cv2.CALIB_ZERO_TANGENT_DIST
    | cv2.CALIB_FIX_K1
    | cv2.CALIB_FIX_K2
    | cv2.CALIB_FIX_K3
    | cv2.CALIB_FIX_K4
    | cv2.CALIB_FIX_K5
    | cv2.CALIB_FIX_K6
)
truepoints = np.array(truepoints, dtype=np.float32)
# truepoints = truepoints.reshape((-1,1,3)).astype(np.float32)

ipoints = np.array(ipoints, dtype=np.float32)
# ipoints = ipoints.reshape((-1,1,3)).astype(np.float32)

# print(truepoints.shape)
# print(ipoints.shape)

for el in unusable:
    print(el)
print(
    f"Used {usableImageCount}/{len(calibration_images)} images.  See above for list of unusable images."
)

retval, K, dcoeffs, rvecs, tvecs = cv2.calibrateCamera(
    truepoints,
    ipoints,
    imagesize,
    cameraMatrix=None,
    distCoeffs=None,
    flags=flags,
)
print(K)
print(dcoeffs)
fx = K[0, 0]
fy = K[1, 1]
cx = K[0, 2]
cy = K[1, 2]

params = (fx, fy, cx, cy)

print()
print("all units below measured in pixels:")
print("  fx = {}".format(K[0, 0]))
print("  fy = {}".format(K[1, 1]))
print("  cx = {}".format(K[0, 2]))
print("  cy = {}".format(K[1, 2]))
print()
print("pastable into Python:")
print("  fx, fy, cx, cy = {}".format(repr(params)))
print()

loaded testimages/cal2.jpg of size 1080x1920
could not find all apriltags
loaded testimages/cal3.jpg of size 1080x1920
loaded testimages/cal4.jpg of size 1080x1920
loaded testimages/cal5.jpg of size 1080x1920
could not find all apriltags
loaded testimages/cal6.jpg of size 1080x1920
loaded testimages/cal7.jpg of size 1080x1920
loaded testimages/cal8.jpg of size 1080x1920
loaded testimages/cal9.jpg of size 1080x1920
could not find all apriltags
loaded testimages/cal10.jpg of size 1080x1920
loaded testimages/cal11.jpg of size 1080x1920
could not find all apriltags
loaded testimages/cal12.jpg of size 1080x1920
loaded testimages/cal13.jpg of size 1080x1920
loaded testimages/cal14.jpg of size 1080x1920
could not find all apriltags
loaded testimages/cal15.jpg of size 1080x1920
loaded testimages/cal16.jpg of size 1080x1920
44/45 testimages/cal2.jpg
39/45 testimages/cal5.jpg
46/45 testimages/cal9.jpg
46/45 testimages/cal11.jpg
46/45 testimages/cal14.jpg
Used 10/15 images.  See above for list of

Given 4 points in one frame, and their corresponding points in another, calculate the transformation matrix between the frames

In [3]:
from matplotlib import gridspec
import marthabot.utils.realsense_utils as rsu 
import numpy as np
import cv2 as cv

pipe = rsu.get_pipe()


In [4]:

grid_size = (6,9)
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((grid_size[0]*grid_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:grid_size[0],0:grid_size[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
pret, pmtx, pdist, prvecs, ptvecs = 0,0,0,0,0

In [9]:
while True:
    print("Getting more images to improve estimates")
    i = 0
    while i < 5:
        frame, _ = rsu.get_frame(pipe)

        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(frame, (grid_size[0],grid_size[1]), None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            i += 1
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(frame,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            cv.drawChessboardCorners(frame, (grid_size[0],grid_size[1]), corners2, ret)
        cv.imwrite("output.jpg",frame)

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frame.shape[::-1], None, None)
    # print(mtx)
    # print(dist)
    evaluatat = (cv.absdiff(mtx,pmtx)) 
    print(evaluatat)
    if (evaluatat < 1).all():
        break
    else:
        pret, pmtx, pdist, prvecs, ptvecs = ret, mtx, dist, rvecs, tvecs
    


Getting more images to improve estimates


[[0.41717426 0.         1.35858572]
 [0.         0.11615335 0.30910743]
 [0.         0.         0.        ]]
Getting more images to improve estimates


KeyboardInterrupt: 

In [55]:
img = cv2.imread('output.jpg')
cv2.imwrite('pre.png', img)
h, w = img.shape[:2]
print(h,w)
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# print(dst)

# crop the image
x, y, w, h = roi
# dst = dst[y:y+h, x:x+w]
print(dst.shape)
print(img.shape)
cv2.imwrite('post.png', dst)
diff = cv2.absdiff(img,dst)
cv2.imwrite('diff.png', diff)


720 1280
[[[135 135 135]
  [137 137 137]
  [138 138 138]
  ...
  [136 136 136]
  [136 136 136]
  [137 137 137]]

 [[137 137 137]
  [138 138 138]
  [137 137 137]
  ...
  [136 136 136]
  [135 135 135]
  [136 136 136]]

 [[136 136 136]
  [136 136 136]
  [135 135 135]
  ...
  [137 137 137]
  [136 136 136]
  [137 137 137]]

 ...

 [[ 14  14  14]
  [ 13  13  13]
  [ 13  13  13]
  ...
  [ 42  42  42]
  [ 44  44  44]
  [ 46  46  46]]

 [[ 16  16  16]
  [ 14  14  14]
  [ 13  13  13]
  ...
  [ 41  41  41]
  [ 43  43  43]
  [ 45  45  45]]

 [[ 17  17  17]
  [ 14  14  14]
  [ 13  13  13]
  ...
  [ 41  41  41]
  [ 42  42  42]
  [ 43  43  43]]]
(720, 1280, 3)
(720, 1280, 3)


True

In [22]:
def rt2orientation(rt):
    r13 = rt[0,2]
    print(r13)
    r23 = rt[1,2]
    r33 = rt[2,2]

    r11 = rt[0,0]
    r12 = rt[0,1]

    ry = np.arcsin(-r13) * 180 / np.pi
    rz = np.arctan2(r12,r11) * 180 / np.pi
    rx = np.arctan2(r23,r33) * 180 / np.pi
    return (rz,ry,rx)

calc1 = np.array([ 942.763,  373.522,  19.419,  1.000])
real1 = np.array([ 945.000,  370.000,  0.000,  1.000])
calc2 = np.array([ 893.323,  374.951,  29.269,  1.000])
real2 = np.array([ 895.000,  370.000,  0.000,  1.000])
calc3 = np.array([ 215.177,  215.715,  80.537,  1.000])
real3 = np.array([ 200.000,  200.000,  0.000,  1.000])
calc4 = np.array([ 312.455,  167.234,  36.752,  1.000])
real4 = np.array([ 300.000,  200.000,  0.000,  1.000])
# calc4 = np.array([ 843.701,  382.801,  35.101,  1.000])
# real4 = np.array([ 845.000,  370.000,  0.000,  1.000])

u = np.array([calc1,calc2,calc3,calc4]).reshape(4,4)
v = np.array([real1,real2,real3,real4]).reshape(4,4)

# print(u)
print(v)
print(np.linalg.inv(u))
print(u @ np.linalg.inv(u))
t = v @ np.linalg.inv(u)

print(t)
print(rt2orientation(t))


[[945. 370.   0.   1.]
 [895. 370.   0.   1.]
 [200. 200.   0.   1.]
 [300. 200.   0.   1.]]
[[  0.35725949  -0.37293477   0.07769117  -0.06201589]
 [ -0.93323486   0.97881147  -0.20215402   0.15657741]
 [  1.82705603  -1.91234687   0.41928219  -0.33399135]
 [-22.70687854  23.117348    -5.87742858   6.46695912]]
[[ 1.00000000e+00 -3.55271368e-15  8.88178420e-16  1.77635684e-15]
 [ 6.39488462e-14  1.00000000e+00  7.99360578e-15  0.00000000e+00]
 [ 1.42108547e-14  3.55271368e-15  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.48689958e-14  4.44089210e-15  1.00000000e+00]]
[[ -30.39355841   32.85423475   -7.25625921    5.79558286]
 [ -48.25653302   51.50097329  -11.14081772    8.89637746]
 [-137.90195308  144.29268848  -30.76999805   25.37926266]
 [-102.17600386  106.99921141  -23.00088103   19.17767347]]
-7.256259205026308
(132.77201238037787, nan, -160.09632231621543)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in arcsin
  # Remove the CWD from sys.path while we load stuff.
